In [5]:
import datetime
from fileinput import filename
import io
import os
import warnings

import dash
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
from collections import OrderedDict

from data_processing_helper import *
warnings.filterwarnings("ignore")

In [6]:
sorted(os.listdir('oxy_c14_lga14'))

['MTF134BC14HSOXYLGA14_output.xlsx',
 'MTF134CC14HSOXYLGA14_output.xlsx',
 'MTF134DC14HSOXYLGA14_output.xlsx',
 'MTF134EC14HSOXYLGA14_output.xlsx']

In [14]:
filepath = 'oxy_c14_lga14/MTF134EC14HSOXYLGA14_output.xlsx'
df = cleanup(filepath)
df_time = df.copy()
cols = df_time.columns.tolist()
cutoff = cols.index('Active 1')
time_begin = df_time['Start Datetime'].min().time()
time_to_minus = datetime.timedelta(hours=time_begin.hour,
                                minutes=time_begin.minute,
                                seconds=time_begin.second)

def transform_to_time(row):
    start_time = row['Start Datetime']
    for col in cols[cutoff:]:
        secs = row[col]
        if secs != 0:
            row[col] = start_time + datetime.timedelta(seconds=secs) - time_to_minus
        else:
            pass
    return row

df_time = df_time.apply(transform_to_time,axis=1)
numSubjects = len(df_time)
stats = []
for i in range(numSubjects):
    test_row = df_time.iloc[i]
    for col in cols[cutoff:]:
        stat = []
        if (type(test_row[col]) != float) and (type(test_row[col]) != int):
            stat.append(test_row['Subject'])
            stat.append(test_row[col])
            stat.append(test_row[col] + datetime.timedelta(seconds=20))
            stat.append(col.split(' ')[0])
            stats.append(stat)
        else:
            pass

timeline_data = pd.DataFrame(stats,columns = ['Subject','Start','End','Type'])
timeline_graph = px.timeline(timeline_data, 
                         x_start="Start", 
                         x_end="End", 
                         y="Subject",
                         color='Type',
                         title='Actions Timeline')

timeline_graph.update_xaxes(tickformat="%H:%M:%S",
                            tickformatstops=[dict(dtickrange=[1800000, 10800000], value="%H:%M")])

timeline_graph.update_yaxes(categoryorder="category ascending")

timeline_graph.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                             'paper_bgcolor': 'rgba(0, 0, 0, 0)'})

oxy_c14_lga14/MTF134EC14HSOXYLGA14_output.xlsx
CLEANING COMPLETED
